In [ ]:
import htcondor
import ipywidgets as widgets
from IPython.display import display

# 1) Build the input widgets, entered
exe = widgets.Text('my_job.sh', description='Executable:')
exe.layout = widgets.Layout(width='60%')
exe.style.description_width = '35%'

args = widgets.Text('', description='Arguments:')
args.layout = widgets.Layout(width='60%')
args.style.description_width = '35%'

cpus = widgets.IntSlider(value=1, min=1, max=16, description='CPUs:')
cpus.layout = widgets.Layout(width='60%')
cpus.style.description_width = '35%'

mem = widgets.IntText(value=1024, description='Memory (MB):')
mem.layout = widgets.Layout(width='60%')
mem.style.description_width = '35%'

submit_btn = widgets.Button(description='Submit Job', button_style='primary')
status_out = widgets.Output()

# 2) Pack status into its own box for spacing
status_box = widgets.VBox([status_out])
status_box.layout = widgets.Layout(width='60%',
                                   min_height='4rem',
                                   margin='1rem 0 0 0',
                                   justify_content='flex-start',
                                   align_items='flex-start',
                                   border='1px solid #ddd',
                                   padding='0.5rem',
                                   border_radius='4px')

# 3) Create the card container, center everything vertically/horizontally
card = widgets.VBox(
    [
      widgets.HTML("<h1 style='text-align:center;margin-bottom:0.5rem;'>HTCondor Submission</h1>"),
      widgets.HTML("<p style='text-align:center;color:#555;margin-top:0;margin-bottom:1.5rem;'>Fill in the fields below and click Submit.</p>"),
      exe, args, cpus, mem,
      submit_btn,
      status_box
    ],
    layout=widgets.Layout(
        width='80%',
        max_width='800px',
        height='80vh',             # stretch vertically
        padding='2%',
        border='1px solid #ddd',
        border_radius='8px',
        display='flex',
        flex_flow='column',
        align_items='center',      # center children horizontally
        justify_content='flex-start', 
        overflow='auto'
    )
)

# 4) Full‑screen container to center the card
container = widgets.Box(
    [card],
    layout=widgets.Layout(
        display='flex',
        align_items='center',
        justify_content='center',
        height='100vh',
        width='100vw',
        padding='1rem'
    )
)

display(container)

# 5) Hook up submission logic
def on_submit(b):
    b.disabled = True
    status_out.clear_output()
    with status_out:
        print("Queuing…")
    try:
        schedd = htcondor.Schedd()
        desc = htcondor.Submit({
            'executable': exe.value,
            'arguments': args.value,
            'request_cpus': str(cpus.value),
            'request_memory': f"{mem.value}MB",
            'output': '$(Cluster).$(Process).out',
            'error':  '$(Cluster).$(Process).err',
            'log':    '$(Cluster).log'
        })
        with schedd.transaction() as txn:
            cid = desc.queue(txn)
        with status_out:
            print(f"Submitted as cluster {cid}")
    except Exception as e:
        with status_out:
            print(f" Failed: {e}")
    finally:
        b.disabled = False

submit_btn.on_click(on_submit)


